In [ ]:
import constants
import pandas  as pd
from tqdm import tqdm
import hashlib
import math
from pandarallel import pandarallel
import random

In [ ]:
extended = True
constants.set_crawl_source(constants.CRAWL_SOURCE_EXTENDED if extended else constants.CRAWL_SOURCE_CHICAGO)

In [ ]:
df = pd.read_pickle(constants.LONG_DATA_FILE)

In [ ]:
business_df = pd.read_pickle(constants.BUSINESS_DATA_FILE)

In [ ]:
df_last = df[df.crawl_id == constants.CRAWL_ORDER[-1]]

In [ ]:
rand = random.Random(0)
sample = random.choices(df_last.business_id.unique(),k=100)

In [ ]:
business_df["stripped_url"] = business_df["url"].apply(lambda s: s.split("?")[0])

In [ ]:
business_df["stripped_url"].iloc[0]

In [ ]:
df[df.content.str.startswith("""MUST VISIT!! This place has just opened up and I'll tell you what, it's DIVINE!""")].content.iloc[0]

In [ ]:
rand = random.Random(1)
for bid in sample:
    
    #Assume we have the same number of reviews, pick a random review in there
    
    df_bids = df_last[df_last.business_id == bid]
    num_rec = len(df_bids[df_bids.flagged == False])
    num_notrec = len(df_bids[df_bids.flagged == True])
    
    #Pick recommended or not recommended?
    rec = rand.random() <= num_rec / (num_rec + num_notrec)
    
    offset = rand.randint(1,num_rec if rec else num_notrec)-1
    if rec:
        url = f"{business_df[business_df['id'] == bid].stripped_url.iloc[0]}?start={offset//10 * 10}"
    else:
        url = f"{business_df[business_df['id'] == bid].stripped_url.iloc[0].replace('/biz/','/not_recommended_reviews/')}?not_recommended_start={offset//10}"
    
    print(f"{bid},{offset%10 + 1},{url}")
    

In [ ]:
business_df[business_df['id'] == bid]

In [ ]:
business_df

In [ ]:
bid

In [ ]:
validation = pd.read_csv("../../data/validation/yelp_udis_crawl_check.csv",names=["bid","offset","url","content"])

In [ ]:
successes = 0
total = 0
for _,row in validation[validation.content.notnull()].iterrows():
    bid = row.bid
    text = row.content
    
    lines = [line for line in text.split("\n") if line]
    
    df_last_bid = df_last[(df_last.business_id == bid)]
    df_last_bid["content"] = df_last_bid.content.apply(lambda s: s.replace(u'\xa0', u' '))
    refined = df_last_bid[df_last_bid.content.str.contains(lines[0][:10],case=False,regex=False)]
    if len(refined) != 0:
        for line in lines:
            line = line.strip()
            refined2 = refined[refined.content.str.contains(line,case=False,regex=False)]
            if len(refined2) == 0:
                print(refined.content.iloc[0])
                print("----")
                print(line)
                print("--------------------------------------------")
                refined = refined2
                #raise
                break
            refined = refined2
    else:
        print(f"Failure: {row.bid},{row.content},{row.url}")
    success = len(refined) != 0
    if success:
        successes += 1
    total += 1
    
print(f"{successes/total:%}")
                
            
    
    

In [ ]:
df[df.business_id == "=-HuWZFXXqKrIuhPTlmFRtQ"]

In [ ]:
line

In [ ]:
line in refined.content.iloc[0]

In [ ]:
refined.content.iloc[0]